# Introduction and Overview of existing gradient algorithms

In this assignment, we explore the evolution and significance of gradient descent algorithms, focusing on their applications in handling complex data-driven problems prevalent in fields such as machine learning and natural language processing. We will delve into the foundations of both classical and adaptive stochastic gradient techniques and investigating their convergence properties.

### Historical Context

Gradient descent algorithms have evolved significantly, starting from the stochastic approximation methods of Kiefer-Wolfowitz and Robbins-Monro in the 1950s, to the introduction of advanced techniques like Momentum Gradient Descent and Nesterov's accelerated method in the 1980s. The 2010s marked a shift towards adaptive methods, with algorithms like AdaGrad, RMSProp, and ADAM, each bringing unique approaches to learning rate adjustments and showcasing effectiveness in various applications, particularly in deep learning.

### Learning Objectives

1. Understand the fundamental concepts of gradient, gradient descent, and stochastic optimization;
2. Explore the theoretical foundations and practical applications of various stochastic gradient descent algorithms;
3. Compare the performance of different gradient descent algorithms on a test convex and smooth objective function.

## Prerequisites

Before delving into the implementation and comparison of gradient descent algorithms, it is essential to set up the necessary computational environment. We will be utilizing the PyTorch library to perform all calculations, as it offers a flexible and efficient platform for scientific computing, particularly in machine learning.

In [ ]:
# Import the PyTorch library
import torch

# Import typing annotations for assignment hints
from typing import Callable

# Check the version of PyTorch
print("PyTorch Version:", torch.__version__)

# Perform a basic operation to test PyTorch
a = torch.tensor([1.0, 2.0])
b = torch.tensor([3.0, 4.0])

# Assert the result of the sum
assert torch.equal(a + b, torch.tensor([4.0, 6.0])), "The sum of a and b is incorrect!"

### Algorithm convergence visualisation

Iterations of the each **gradient descent** algorithm can be plotted as the path on 3D surface, in case model has only 2 independent parameters $\{ \theta_{1}, \theta_{2} \}$ so they can represent $X$ and $Y$ axises, and remaining $Z$ axis represents value of estimation equation to optimize $J(\theta_{1}, \theta_{2})$. All visualisation tools are used from [matplotlib](https://github.com/matplotlib/matplotlib) package.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as grid
import matplotlib.cm as colormaps
from matplotlib import rcParams, cycler

from mpl_toolkits.mplot3d.axes3d import Axes3D

To plot the projection of the 3D surface of $J(\theta_{i})$ will be used a custom procedure.

In [ ]:
def plot_grid(F: Callable[[torch.Tensor, torch.Tensor], torch.Tensor],
              X: torch.Tensor, Y: torch.Tensor,
              elev: int = 30, azim: int = 50, ax=None) -> Axes3D:
  """
  Plots 3D surface grid for 2 independent parameters and estimation equation.

  Args:
      F (Callable[[torch.Tensor, torch.Tensor], torch.Tensor]): Estimation equation.
      X (torch.Tensor): First independent parameter.
      Y (torch.Tensor): Second independent parameter.
      elev (int, optional): Vertical rotation angle. Defaults to 30.
      azim (int, optional): Horizontal rotation angle. Defaults to 50.
      ax (Axes3D, optional): Predefined plotting axis. If None, a new one will be created. Defaults to None.

  Returns:
      Axes3D: Generated plotting axis for potential reusability.
  """

  # Generating grid
  x, y = torch.meshgrid(X, Y, indexing='xy')

  # If grid plotting axis is not defined above, the new one will be created
  if ax is None:
      fig = plt.figure()
      ax = fig.add_subplot(111, projection='3d')
      ax.view_init(elev=elev, azim=azim)

  # Plotting grid
  surf = ax.plot_surface(x.numpy(), y.numpy(), F(x, y).numpy(),
                          cmap=colormaps.coolwarm,
                          antialiased=True)
  fig.colorbar(surf)

  # For axis reusability purposes
  return ax

## Stochastic Optimization Problem

Stochastic optimization problems form the bedrock for addressing uncertainties and randomness inherent in various domains like finance, machine learning, and operations research. Contrasting with deterministic optimization, where the objective function and constraints are well-defined, stochastic optimization introduces challenges by incorporating components that exhibit randomness. In this section, we delve into the mathematical formulation of a stochastic optimization problem and explore how stochastic gradient descent algorithms tackle the challenges presented by this formulation.

### Mathematical Formulation

Given an objective function $ f: X \to \mathbb{R} ^ n $ with domain $ X \subset \mathbb{R} ^ n $, and a convex and differentiable function $ F: X \times \Xi \to \mathbb{R} ^ 1 $ that depends on the determined variable $ x \in X $ and a stochastic variable $ \xi \in \Xi $, defined on a space $ (\Xi, \Sigma, P) $, a stochastic optimization problem can be represented as:

$$
\min_{x \in X} \left[f(x) = \mathbb{E} F(x, \xi) = \int_{\xi \in \Xi} F(x, \xi) P(d \xi), X \subset \mathbb{R} ^ n\right]
$$

Here, $ \mathbb{E} $ denotes the mathematical expectation. The intrinsic challenge of this problem lies in the difficulty of explicitly calculating the value of an integral (mathematical expectation) and the gradient of this integral. Stochastic gradient descent algorithms, leveraging gradients $ \nabla_{x} F(x, \xi) $ of a stochastic function $ F(\cdot, \xi) $ or their finite-difference counterparts, offer a solution to this challenge.

### Practical Implications

These optimization problems are pivotal in scenarios where decision-making is dependent on incomplete or uncertain information. Employing techniques such as random sampling, Monte Carlo simulations, and stochastic gradients, stochastic optimization methods effectively and efficiently traverse the optimization landscape, aiming for convergence to the optimal solution.

### Stochastic logistic regression

Logistic regression is chosen as an example of an optimization problem for comparing the convergence rate of the gradient descent algorithms. This mathematical model can be described as a binary classifier, which outputs a probability value of a certain set of features $ x_{i} $ to belong to a certain class. Using the definition of optimization problem from above, the classifier can be represented in the form of two components: an adder that combines all the characteristics into a single one: $ z_{j} = \sum_{i=1}^{n} \theta_{i} x_{i, j} + \varepsilon $ (or $ z_{j} = \theta^{T} \cdot x + \varepsilon $), and a converter that calculates the probability of the characteristics belonging to a certain class based on the output value of an adder: $g(a) = \frac{1}{1 + e^{-a}}$. Here $ \varepsilon \simeq N(0, 1) $ is a stochastic parameter in the model.

In [ ]:
def adder_fn(theta: torch.Tensor, x: torch.Tensor) -> torch.Tensor:
    return theta.t() @ x

def converter_fn(a: torch.Tensor) -> torch.Tensor:
    return 1.0 / (1.0 + torch.exp(-a))

# x = (0, 0), theta = (1, 1) => g = 0.5
assert torch.allclose(
    converter_fn(adder_fn(torch.tensor([[1.0], [1.0]]),
                          torch.tensor([[0.0], [0.0]]))),
    torch.tensor([[0.5]]))

# x = (1, 0), theta = (1, 1) => g = 0.7310586
assert torch.allclose(
    converter_fn(adder_fn(torch.tensor([[1.0], [1.0]]),
                          torch.tensor([[1.0], [0.0]]))),
    torch.tensor([[0.7310586]]))

# x = (0, -1), theta = (1, 1) => g = 0.26894143
assert torch.allclose(
    converter_fn(adder_fn(torch.tensor([[1.0], [1.0]]),
                          torch.tensor([[0.0], [-1.0]]))),
    torch.tensor([[0.26894143]]))

# x = (0.5, -1), theta = (2, -1.5) => g = 0.9241418
assert torch.allclose(
    converter_fn(adder_fn(torch.tensor([[2.0], [-1.5]]),
                          torch.tensor([[0.5], [-1.0]]))),
    torch.tensor([[0.9241418]]))

The solution to the problem is to find optimal $\theta_{i}$ so for input features $x_{i}$ the **loss** $J(\theta_{i})$ between output value of classifier $\hat{y_{j}} = g(z_{\theta}(x_{i, j}))$ and expected value $y_{j}$ will be minimal.

The set of classes, mentioned above, can be represented as $\{ 0, 1 \}$, e. g. if model output is $1$ - features belong to the class, and if $0$ - features do not belong to the class. In this case, the rule of the fitting regression model can be interpreted as follows: *if expected output is $y=1$, the loss should approach $0$ when model output approaches $1$ and grow to infinity when model output approaches $0$. For the case $y=0$ these conditions apply vise versa.*

Model fitting rule or **loss function** can expressed as limits:

$\begin{cases}
\lim_{g(z_{\theta}(x_{i, j})) \to 1} J_{j}(\theta_{i}) = 0, & \lim_{g(z_{\theta}(x_{i, j})) \to 0} J_{j}(\theta_{i}) = \infty, & y=1 \\
\lim_{g(z_{\theta}(x_{i, j})) \to 0} J_{j}(\theta_{i}) = 0, & \lim_{g(z_{\theta}(x_{i, j})) \to 1} J_{j}(\theta_{i}) = \infty, & y=0
\end{cases}$

Limitation conditions fit the $\log(x)$ function, so the **loss function** can be expressed as the following system:

$ J_{j}(g, y) = \begin{cases}
-\log(g), & y = 1  \\
-\log(1 - g), & y = 0
\end{cases}$

The statement can be joined into a single formula: $ J_{j}(g, y) = - y \log(g) - (1 - y) \log(1 - g) $ - this formula is appliable for single pair of features and output. For multiple pairs loss value can be calculated as the mean of losses for single pairs: $ J(g, y) = \frac{1}{m} \sum_{j=1}^{m} J_{j}(g, y) = \frac{1}{m} \sum_{j=1}^{m} ( - y \log(g) - (1 - y) \log(1 - g) ) = -\frac{1}{m} \sum_{j=1}^{m} (y \log(g) + (1 - y) \log(1 - g) )$

In [ ]:
def loss_fn(g: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
    """
    Computes the binary cross-entropy loss between predicted probabilities and target labels.

    Args:
    g (torch.Tensor): The predicted probabilities.
    y (torch.Tensor): The target labels.

    Returns:
    torch.Tensor: The computed binary cross-entropy loss.
    """

    return -y * torch.log(g) - (1 - y) * torch.log(1 - g)

As the result, the derived fitting rule (**loss function**) is a convex function with only one minimum for each output class $ \{ 0, 1 \} $. This can be demonstraited on the following plots:

In [ ]:
# Generate an array of values
g = torch.arange(0.001, 1.0, 0.001)

# Compute JLeft and JRight
loss_left = -torch.log(g)
loss_right = -torch.log(1 - g)

# Plotting
plt.plot(g.numpy(), loss_left.numpy(), lw=5, label='Left limit condition `y = 1`.')
plt.plot(g.numpy(), loss_right.numpy(), lw=5, label='Left limit condition `y = 0`.')
plt.xlabel('Converter output value')
plt.ylabel(r'An objective function $ J(\theta) $ value')
plt.grid(True)
plt.legend(loc='upper center')

plt.show()

The loss function is formed from combined limit conditions, which is plotted as a diagonal cross-section of the grid below:

In [ ]:
# Create a tensor y, ranging from 0.001 to 1.0 with steps of 0.001
y = torch.arange(0.001, 1.0, 0.001)

# Arguments for output equals 0
yleft = torch.zeros(y.shape[0])

# Arguments for output equals 1
yright = torch.ones(y.shape[0])

# Loss function plot in 3D cross-section
ax = plot_grid(loss_fn, g, y, elev=30, azim=70)

# Loss function plot for output equals 0
ax.plot(g.numpy(), yleft.numpy(), loss_fn(g, yleft).numpy(), lw=5, label="loss for y=0")

# Loss function plot for output equals 1
ax.plot(g.numpy(), yright.numpy(), loss_fn(g, yright).numpy(), lw=5, label="loss for y=1")

plt.legend()
plt.show()

Loss calculation test cases:

In [ ]:
assert torch.allclose(
    loss_fn(torch.tensor([0.0001]), torch.tensor([0.0])),
    torch.tensor([0.00010002]))

assert torch.allclose(
    loss_fn(torch.tensor([0.9998]), torch.tensor([1.0])),
    torch.tensor([0.00019999]))

assert torch.allclose(
    loss_fn(torch.tensor([0.8]), torch.tensor([0.0])),
    torch.tensor([1.609438]))

assert torch.allclose(
    loss_fn(torch.tensor([0.2]), torch.tensor([1.0])),
    torch.tensor([1.609438]))

### Gradient approximation

While libraries like PyTorch offer automatic differentiation, this assignment encourages a hands-on approach. We will be utilizing the second-order accurate central differences method to estimate gradients, offering insight into the intricacies of gradient computation and its role in optimization algorithms.

We can derive approximation formula from the Taylor's series polynomial while discarding unnecessary residual terms that have higher accuracy order than the first order:

$$ f(x_0 + h) = f(x_0) + f'(x_0)(h) + o(h) $$

By applying finite differences approximation we get left and right finite differences:

$$
\begin{cases}
  f'_{-}(x) = \frac{f(x + h) - f(x)}{h} - \frac{h f''(\xi)}{2} \\
  f'_{+}(x) = \frac{f(x) - f(x - h)}{h} + \frac{h f''(\xi)}{2}
\end{cases}
$$

The accuracy of the approximation depends on the number of nodes on the numerical partitioning grid, thus the smaller step difference, the higher precision. Using the Runge-Romberg-Richardson algorithm we can achieve an increase in the order of the precision of the partitioning grid up to $ O(h^2) $ without adding extra iterations to the approximation algorithm:

$$
\begin{cases}
  f'_{-}(x) = \frac{-3 f(x) + 4 f(x + h) - f(x + 2h)}{2h} + \frac{h^2 f'''(\xi)}{3} \\
  f'(x) = \frac{f(x + h) - f(x - h)}{2h} + \frac{h^2 f'''(\xi)}{6} \\
  f'_{+}(x) = \frac{f(x - 2h) - 4 f(x - h) + 3 f(x)}{2h} + \frac{h^2 f'''(\xi)}{3}
\end{cases}
$$

The approach of approximating the gradient value of the target function with a finite-difference schema lets us generalize optimization problems on any kind of analytical functions.

In [ ]:
def grad_left(F: Callable[[torch.Tensor], torch.Tensor], x: torch.Tensor, h: float = 0.001) -> torch.Tensor:
  """A finite-difference approximation for left-side gradient ∇F₋(x) with the precision order O(h^2).

  Args:
      F (Callable[[torch.Tensor], torch.Tensor]): an objective function F(x) with a single input argument x ∈ ℝⁿ.
      x (torch.Tensor): an input vector x ∈ ℝⁿ, where the derivative is calculated.
      h (float, optional): a step of the derivative partitioning grid with the range of 0<h<1. The lower value, the higher gradient precision. Defaults to 0.001.

  Returns:
      torch.Tensor: a gradient vector approximation ∇F₋(x).
  """

  pass # TODO: Implement second-order accurate forward difference algorithm


def grad_center(F: Callable[[torch.Tensor], torch.Tensor], x: torch.Tensor, h: float = 0.001) -> torch.Tensor:
  """A finite-difference approximation for central gradient ∇F(x) with the precision order O(h^2).

  Args:
      F (Callable[[torch.Tensor], torch.Tensor]): a target function F(x) with a single input argument x ∈ ℝⁿ.
      x (torch.Tensor): an input vector x ∈ ℝⁿ, where the derivative is calculated.
      h (float, optional): a step of the derivative partitioning grid with the range of 0 < h < 1. The lower value, the higher gradient precision. Defaults to 0.001.

  Returns:
      torch.Tensor: a gradient vector approximation ∇F(x).
  """

  pass # TODO: Implement second-order accurate center difference algorithm


def grad_right(F: Callable[[torch.Tensor], torch.Tensor], x: torch.Tensor, h: float = 0.001) -> torch.Tensor:
  """A finite-difference approximation for right-side gradient ∇F+(x) with the precision order O(h^2).

  Args:
      F (Callable[[torch.Tensor], torch.Tensor]): a target function F(x) with a single input argument x ∈ ℝⁿ.
      x (torch.Tensor): an input vector x ∈ ℝⁿ, where the derivative is calculated.
      h (float, optional): a step of the derivative partitioning grid with the range of 0<h<1. The lower value, the higher gradient precision. Defaults to 0.001.

  Returns:
      torch.Tensor: a gradient vector approximation ∇F+(x).
  """

  pass # TODO: Implement second-order accurate center difference algorithm

We will check the accuracy of the implemented method on the test case:

$ F(x, y) = x^2 + xy + y^2 \implies \begin{cases} \frac{\partial F(x, y)}{\partial x} = 2x + y, \frac{\partial F(2.0, -1.0)}{\partial x} = 3.0 \\ \frac{\partial F(x, y)}{\partial y} = x + 2y, \frac{\partial F(2.0, -1.0)}{\partial y} = 0.0
 \end{cases} $

In [ ]:
h = 0.001
x = torch.tensor([2.0, -1.0])
f = lambda x: x[0] ** 2 + x[0] * x[1] + x[1] ** 2

assert torch.allclose(grad_left(f, x, h=h), torch.tensor([3.0, 0.0]), rtol=h)
assert torch.allclose(grad_center(f, x, h=h), torch.tensor([3.0, 0.0]), rtol=h)
assert torch.allclose(grad_right(f, x, h=h), torch.tensor([3.0, 0.0]), rtol=h)